In [1]:
import os, sys, glob

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.signal as signal

import mne

import tensorpac
import pactools

In [2]:
# Set subject ID
subj_id = 10

print(os.listdir('../'))
outputs_path = '../outputs'
data_path = '../data'
meg_dir = os.path.join(data_path, f'subj_{subj_id}', 'meg')
mri_dir = os.path.join(data_path, f'subj_{subj_id}', 'mri')
fs_subjs_dir = os.path.join(data_path, 'fs_subjects_dir')

['.DS_Store', 'codes', 'data', 'plots_molly', 'outputs']


In [3]:

def mk_epochs_new(meg,  mod_freq=None, tmin=None, tmax=None, baseline=None, annot_pattern='', new_event_value=100):
    """This function creates epochs based on specified mod_freq and annotation_pattern
    Arguments:
        meg: annotated MNE object, where bad time spans are annotated as BAD_*
        tmin: start time of the epoch in seconds. This parameter should match with baseline.
        baseline: to specify baseline correction. e.g. tmin=0, baseline=(0, 0) applies no baseline correction.
        mod_freq: to specify the modulating frequency of interest. This parameter should match with the annot_pattern
        annot_pattern: The annotation pattern, based on which epochs are created.
        new_event_value: to specify new label for events. The default is 100. Optional argument.
    Returns:
        epoch: Epoch MNE object.
    Example1:
        The following call will create epochs from mod_freq=1 and encoding events, and will apply baseline correction
        between -0.5 and 0. The length of created epochs would be 8/mod_freq + abs(tmin) = 8 + 0.5 = 8.5 seconds or
        8.5 * sfreq = 8.5 * 300 samples
        mk_epochs(meg.copy(), mod_freq=1., tmin=-0.5, baseline=(-0.5, 0), annot_pattern='e/1.0/')
    Example2:
        create epochs from maintenance events with mod_freq=3.5
        mk_epochs(meg.copy(), mod_freq=3.5, tmin=0, baseline=(0, 0), annot_pattern='m/3.5/')
    Notes:
        tmin and annotation patterns should match.
        baseline = (None, 0) sets baseline to MNE defaults.
        baseline = (0, 0), tmin = 0 sets to no baseline.
        baseline = (-0.5, 0), tmin = -0.5
    """
    if not tmax:
        tmax = (8 / mod_freq) - 1 / meg.info['sfreq']
    events = mne.events_from_annotations(meg)
    annot = list(events[1].keys())
    indx_pattern = np.where([annot_pattern in k for k in annot])[0].tolist()
    event_vals_pattern = np.array(list(events[1].values()))[indx_pattern].tolist()
    indx_events = np.where(np.isin(events[0][:, 2], event_vals_pattern))[0].tolist()
    events4epoch = events[0][indx_events, :]
    events4epoch[:, 2] = new_event_value
    annot_epoch = mne.annotations_from_events(events4epoch, meg.info['sfreq'])
    meg.set_annotations(annot_epoch)
    epoch = mne.Epochs(meg, events=events4epoch, tmin=tmin, tmax=tmax, baseline=baseline)
    return epoch

In [4]:
# Load after-ICA MEG data and make epochs
meg = mne.io.read_raw_fif(os.path.join(meg_dir, 'after_ica_meg.fif'))


Opening raw data file ../data/subj_10/meg/after_ica_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 1525967 =      0.000 ...  5086.557 secs
Ready.
Opening raw data file /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/codes/../data/subj_10/meg/after_ica_meg-1.fif...
    Read 5 compensation matrices
    Range : 1525968 ... 2103623 =   5086.560 ...  7012.077 secs
Ready.
Current compensation grade : 3


In [5]:
fm = '3.0'
#epoch = mk_epochs_new(meg.copy(), mod_freq=2., tmin=-0.2, baseline=(-0.2, 0), 
#                   annot_pattern='e/2.0/', new_event_value=101)
epoch = mk_epochs_new(meg.copy(), mod_freq=float(fm), tmin=-0.2, baseline=None, 
                   annot_pattern=f'e/{fm}/', new_event_value=101)
print(epoch.get_data().shape, epoch.info['sfreq'])

Used Annotations descriptions: ['e/1.0/e0/t0/f/r1', 'e/1.0/e0/t0/s/r0', 'e/1.0/e0/t0/s/r1', 'e/1.0/e0/tpi/f/r0', 'e/1.0/e0/tpi/f/r1', 'e/1.0/e0/tpi/s/r0', 'e/1.0/e0/tpi/s/r1', 'e/1.0/epi/t0/f/r0', 'e/1.0/epi/t0/f/r1', 'e/1.0/epi/t0/s/r0', 'e/1.0/epi/t0/s/r1', 'e/1.0/epi/tpi/f/r0', 'e/1.0/epi/tpi/f/r1', 'e/1.0/epi/tpi/s/r0', 'e/1.0/epi/tpi/s/r1', 'e/1.5/e0/t0/f/r0', 'e/1.5/e0/t0/f/r1', 'e/1.5/e0/t0/s/r0', 'e/1.5/e0/t0/s/r1', 'e/1.5/e0/tpi/f/r0', 'e/1.5/e0/tpi/f/r1', 'e/1.5/e0/tpi/s/r0', 'e/1.5/e0/tpi/s/r1', 'e/1.5/epi/t0/f/r1', 'e/1.5/epi/t0/s/r0', 'e/1.5/epi/t0/s/r1', 'e/1.5/epi/tpi/f/r0', 'e/1.5/epi/tpi/f/r1', 'e/1.5/epi/tpi/s/r0', 'e/1.5/epi/tpi/s/r1', 'e/2.0/e0/t0/f/r0', 'e/2.0/e0/t0/f/r1', 'e/2.0/e0/t0/s/r0', 'e/2.0/e0/t0/s/r1', 'e/2.0/e0/tpi/f/r0', 'e/2.0/e0/tpi/f/r1', 'e/2.0/e0/tpi/s/r0', 'e/2.0/e0/tpi/s/r1', 'e/2.0/epi/t0/f/r0', 'e/2.0/epi/t0/f/r1', 'e/2.0/epi/t0/s/r0', 'e/2.0/epi/t0/s/r1', 'e/2.0/epi/tpi/f/r0', 'e/2.0/epi/tpi/f/r1', 'e/2.0/epi/tpi/s/r0', 'e/2.0/epi/tpi/s/r1', '

In [7]:

# Pick Magnetometers
ch_names_mag = [meg.ch_names[j] for j, k in enumerate(meg.get_channel_types()) if k=='mag']
epoch = epoch.load_data().pick_channels(ch_names=ch_names_mag)

Loading data for 72 events and 860 original time points ...
Removing 5 compensators from info because not all compensation channels were picked.


In [8]:
def calc_itc(sig, sfreq):
    """
    This function computes ITPC from epoched time series

    Args:
        sig: epochs x channels x time points
        sfreq: sampling frequency
        
    Returns:
        itpc: Inter-trial phase coherence.
    """
    nepoch, nchan, ntp = sig.shape
    sig = np.reshape(sig, (-1, ntp))  # convert epochs @ channels @ time-points to  (epochs . channels) @ time-points
    nfft = ntp
    xwin = signal.windows.hann(ntp).reshape(1, -1)
    #plt.plot(xwin)
    sig = sig * xwin
    #plt.plot(sig)

    # Compute FFT
    F = scipy.fft.fft(sig)
    xfreq = np.arange(nfft/2) * (sfreq/nfft)
    #xf = fftfreq(ntp, 1/sfreq)[:ntp//2]
    #plt.plot(xfreq, np.abs(F[0,:ntp//2]))

    # Compute power, amplitude, and phase spectra
    AS = 2/xwin.sum() * np.sqrt(F*np.conjugate(F))  # amplitude spectrum
    S = F * np.conjugate(F) / xwin.sum()  # Power spectrum
    P = np.arctan(np.imag(F) / np.real(F))  # Phase spectrum

    # shorten matrix by half
    AS = AS[:, :ntp//2]
    S = S[:, :ntp//2]
    P = P[:, :ntp//2]
    F = F[:, :ntp//2]

    # Reshape Fourier matrix to its to initial shape
    F = F.reshape((nepoch, nchan, ntp//2))

    # inter-trial phase coherence
    itpc = np.squeeze(np.abs(np.mean(F/np.abs(F), axis=0)))  # Channel 68
    # evoked power
    epow = np.squeeze(np.abs(np.mean(F, axis=0)) ** 2)
    # total power
    tpow = np.squeeze(np.mean(np.abs(F) ** 2, axis=0))

    return itpc, xfreq, epow, tpow, F

In [ ]:

# Make epochs
for j, fm in enumerate(['1.0', '1.5', '2.0', '2.5', '3.0', '3.5', '4.0']):
    epoch_0 = prep.mk_epochs(meg.copy(), mod_freq=float(fm), tmin=0, baseline=(0, 0), annot_pattern=f'e/{fm}/e0', new_event_value=101)
    sig_0 = epoch_0.get_data()
    
    epoch_pi = prep.mk_epochs(meg.copy(), mod_freq=float(fm), tmin=0, baseline=(0, 0), annot_pattern=f'e/{fm}/epi', new_event_value=101)
    sig_pi = epoch_pi.get_data()
    #sig_pi = prep.shift_phase_pi(sig_pi, fs=meg.info['sfreq'])
    sig_pi = prep.shift_phase(sig_pi, dphi=np.pi, f=float(fm), fs=meg.info['sfreq'])
    
    #sig_0, sig_pi = sync_sigs(sig_0, sig_pi, float(fm), 300)
    
    sig = np.concatenate((sig_0.copy(), sig_pi.copy()), axis=0)
    #sig = sig_0 # sig_0
    
    # Pick channels with largest auditory response
    if fm == '1.0':
        evoked = epoch_0.average()
        evoked = evoked.copy().filter(l_freq=None, h_freq=20., fir_design='firwin')  # h_freqs = 15, 20, <30
        indx = prep.pick_topchans_erf(evoked=evoked)
        indx30 = indx['indx30_chan']
        indx10lh = indx['indx10lh_chan']
        indx10rh = indx['indx10rh_chan']
    
    sig_sel = sig[:, indx30, :]
    
    fs = meg.info['sfreq']
    itpc, xfreq, epow, tpow, F = calc_itc(sig_sel, fs)
    
    if fm == '1.0':
        fig, ax = plt.subplots(2, 4, sharex=True, sharey=True, figsize=(14, 6))
    ax[j//4, j%4].plot(xfreq, itpc.T)
    ax[j//4, j%4].set_xlim(0.5, 4.2)
    ax[j//4, j%4].set_xticks([1, 1.5, 2, 2.5, 3, 3.5, 4])
    ax[j//4, j%4].set_title(f'{float(fm)}Hz')
    ax[j//4, j%4].grid(True)
    
